# passing stats 

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import requests

from bs4 import BeautifulSoup
import pandas as pd

In [29]:
url = "https://www.pro-football-reference.com/years/"
options = Options()
options.headless = True
b = webdriver.Chrome(options=options)
b.get(url)

In [30]:
page = BeautifulSoup(b.page_source)
table = page.find(id = "div_years")
rows = []
table.find_all("th")
for th in table.find_all("th"):
    for data in th.find_all("a", href = True):
        row = [data.get_text(), data['href']]
        rows.append(row)

links = pd.DataFrame(rows, columns=['year', 'link'])
links

,year,link
0,2020,/years/2020/
1,2019,/years/2019/
2,2018,/years/2018/
3,2017,/years/2017/
4,2016,/years/2016/
...,...,...
96,1924,/years/1924/
97,1923,/years/1923/
98,1922,/years/1922/
99,1921,/years/1921_APFA/


In [74]:
home_url = "https://www.pro-football-reference.com"
def get_data(link):
    url = home_url + link + 'passing.htm'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    tables = soup.find_all("table")
    
    table = tables[0]
    tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                            for row in table.find_all("tr")]
    df = pd.DataFrame(tab_data)
    df.columns = df.iloc[0,:]
    df.drop(index=0,inplace=True)
    df['year'] = link
    
    return df


df_19 = get_data('/years/2019/')
df_18 = get_data('/years/2018/')

data = pd.concat([
    df_19, 
    df_18
]).reset_index(drop=True)

data 

,Rk,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,...,Rate,QBR,Sk,Yds,NY/A,ANY/A,Sk%,4QC,GWD,year
0,1,Jared Goff,LAR,25,QB,16,16,9-7-0,394,626,...,86.5,50.2,22,170,6.90,6.46,3.4,1,2,/years/2019/
1,2,Jameis Winston,TAM,25,QB,16,16,7-9-0,380,626,...,84.3,59.1,47,282,7.17,6.15,7.0,2,2,/years/2019/
2,3,Matt Ryan,ATL,34,QB,15,15,7-8-0,408,616,...,92.1,60.4,48,316,6.25,6.08,7.2,3,2,/years/2019/
3,4,Tom Brady,NWE,42,QB,16,16,12-4-0,373,613,...,88.0,54.5,27,185,6.05,6.24,4.2,1,1,/years/2019/
4,5,Carson Wentz,PHI,27,QB,16,16,9-7-0,388,607,...,93.1,64.8,37,230,5.91,6.26,5.7,2,4,/years/2019/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,102,Logan Thomas,BUF,27,,12,3,,1,1,...,118.7,,0,0,15.00,15.00,0.0,,,/years/2018/
210,103,Colby Wadman,DEN,23,,12,0,,1,1,...,116.7,,0,0,12.00,12.00,0.0,,,/years/2018/
211,104,Tress Way,WAS,28,,16,0,,1,1,...,95.8,,0,0,7.00,7.00,0.0,,,/years/2018/
212,105,Dede Westbrook,JAX,25,wr,16,9,,0,1,...,39.6,10.1,0,0,0.00,0.00,0.0,,,/years/2018/


In [117]:
collection = []
for i in range(0, 30):
    print(i)
    link = links.loc[i]['link']
    df = get_data(link)
    collection.append(df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [118]:
from itertools import chain
list_of_dicts = [cur_df.T.to_dict().values() for cur_df in collection]    
giant_concat_df = pd.DataFrame(list(chain(*list_of_dicts)))

In [121]:
giant_concat_df = giant_concat_df.replace(r'^\s*$', np.NaN, regex=True)
giant_concat_df.to_csv('passing.csv')

giant_concat_df

,Rk,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,...,Y/G,Rate,QBR,Sk,NY/A,ANY/A,Sk%,4QC,GWD,year
0,1,Joe Burrow,CIN,24,qb,8,8,2-5-1,221,330,...,284.0,91.4,60.3,28,5.78,5.77,7.8,NaN,NaN,/years/2020/
1,2,Matt Ryan,ATL,35,qb,8,8,2-6-0,211,316,...,307.8,97.6,75.8,17,7.02,7.20,5.1,NaN,NaN,/years/2020/
2,3,Tom Brady,TAM,43,qb,8,8,6-2-0,204,308,...,273.6,103.1,72.8,10,6.69,7.38,3.1,2,2,/years/2020/
3,4,Carson Wentz,PHI,28,qb,8,8,3-4-1,178,305,...,235.4,73.2,49.1,32,4.94,4.05,9.5,3,2,/years/2020/
4,5,Jared Goff,LAR,26,qb,8,8,5-3-0,186,284,...,268.1,94.6,58.5,10,7.04,7.00,3.4,NaN,NaN,/years/2020/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3051,92,Harry Sydney,SFO,32,NaN,16,0,NaN,0,1,...,0.0,39.6,NaN,0,0.00,0.00,0.0,NaN,NaN,/years/1991/
3052,93,Blair Thomas,NYJ,24,RB,16,12,NaN,1,1,...,1.0,158.3,NaN,0,16.00,36.00,0.0,NaN,NaN,/years/1991/
3053,94,Anthony Thompson,PHO,24,NaN,16,4,NaN,0,1,...,0.0,39.6,NaN,0,0.00,0.00,0.0,NaN,NaN,/years/1991/
3054,95,Al Toon,NYJ,28,WR,15,15,NaN,1,1,...,1.8,118.7,NaN,0,27.00,27.00,0.0,NaN,NaN,/years/1991/


# team offense

In [31]:
home_url = "https://www.pro-football-reference.com"
url = "https://www.pro-football-reference.com/years/2018/#all_team_stats"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find(id = 'team_stats')
tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                                     for row in table.find_all("tr")]
df = pd.DataFrame(tab_data)
df

# def get_team_data(link, table_id):
#     url = home_url + link
#     print(url)
#     page = requests.get(url)
#     soup = BeautifulSoup(page.content, 'html.parser')
#     table = soup.find(id = 'team_stats')
#     print(table)
#     tab_data = [[cell.text for cell in row.find_all(["th","td"])]
#                                     for row in table.find_all("tr")]
#     df = pd.DataFrame(tab_data)


#     df.columns = df.iloc[1,:]
#     df.drop(index=0,inplace=True)
#     df.drop(index=1, inplace=True)

#     return df.reset_index(drop=True)

# get_team_data('/years/2018/', 'team_stats')

AttributeError: 'NoneType' object has no attribute 'find_all'

In [10]:
collection = []
for i in range(0, 30):
    print(i)
    link = links.loc[i]['link']
    df = get_team_data(link, 'team_stats')
    collection.append(df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [12]:
from itertools import chain
def giant_concat(collection):
    list_of_dicts = [cur_df.T.to_dict().values() for cur_df in collection]    
    giant_concat_df = pd.DataFrame(list(chain(*list_of_dicts)))
    giant_concat_df = giant_concat_df.replace(r'^\s*$', np.NaN, regex=True)
    return giant_concat_df

df = giant_concat(collection)
df

,2020,NFL,
0,2019,NFL,"Super Bowl LIV: Kansas City Chiefs (AFC,12-4) ..."
1,2018,NFL,"Super Bowl LIII: New England Patriots (AFC,11-..."
2,2017,NFL,"Super Bowl LII: Philadelphia Eagles (NFC,13-3)..."
3,2016,NFL,"Super Bowl LI: New England Patriots (AFC,14-2)..."
4,2015,NFL,"Super Bowl 50: Denver Broncos (AFC,12-4) defea..."
...,...,...,...
3115,1924,NFL,"Cleveland Bulldogs (NFL,7-1-1)"
3116,1923,NFL,"Canton Bulldogs (NFL,11-0-1)"
3117,1922,NFL,"Canton Bulldogs (NFL,10-0-2)"
3118,1921,APFA,"Chicago Staleys (APFA,9-1-1)"


In [32]:
url = "https://www.pro-football-reference.com/years/2018/#all_team_stats"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [41]:
table = soup.find(id='all_team_stats')
t = table.find(id='team_stats')
tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                                     for row in table.find_all("tr")]
tab_data

[]